In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import calibration as cal
import ATC as atc
import numpy as np
from scipy.special import softmax
import pandas as pd
import os.path
import os

In [19]:
source_probs_fp = "./results/cifar-10-densenet_cifar-10_preds.csv"
source_labels_fp = "./results/cifar-10_labels.csv"

source_probs_df = pd.read_csv(source_probs_fp, header=None)
source_probs = softmax(source_probs_df.to_numpy(), axis = 1)

source_labels_df = pd.read_csv(source_labels_fp, header=None)
source_labels = source_labels_df.to_numpy().reshape(-1)

In [20]:
#print(source_probs.shape)
#print(source_labels.shape)

(10000, 10)
(10000,)


In [21]:
#train calibrator on source validation set
calibration_error = cal.ece_loss(source_probs, source_labels)
print("Calibration error is {}".format(calibration_error))
calibrator = cal.TempScaling(bias=False)
calibrator.fit(source_probs, source_labels)

Calibration error is 0.026556584569866856


In [ ]:
calibrated_source_probs = calibrator.calibrate(source_probs)

In [28]:
test_probs_fp = "./results/cf10-densenet_cf10c-motion_preds.csv"
test_probs_df = pd.read_csv(test_probs_fp, header=None)
test_probs = softmax(test_probs_df.to_numpy(), axis = 1)
print(test_probs.shape)

(50000, 10)


In [29]:
calibrated_test_probs = calibrator.calibrate(test_probs)
#print(np.sum(calibrated_test_probs != test_probs))

In [30]:
atc_acc = atc.ATC_accuracy(calibrated_source_probs, source_labels, calibrated_test_probs)
print(atc_acc)

Threshold is 0.7119902968406677
73.63
